## Load data
Load dataset from desk. These are:
    - X matrix (size trials x neurons) that contains neural responses at different trials.
    - Y choice of high rate (modeled as 1) and low rate (modeled as 0)


In [13]:
import scipy.io as scio
import numpy as np
dirName = '/Users/gamalamin/git_local_repository/Farzaneh/XY.mat';
#dirName = '/home/farznaj/Shares/Churchland/data/fni17/imaging/151022/XY_fni17_151022 XY_lassoSVM.mat';
XY = scio.loadmat(dirName, variable_names=['X', 'Y']);
X = XY.pop('X')
Y = np.squeeze(XY.pop('Y'))
numTrials, numNeurons = X.shape

print ' The data has %d neurons recorded at %d trials' %(numTrials, numNeurons)


 The data has 274 neurons recorded at 446 trials


## Features normalization and scaling
    To removed effects related to scaling and bias of each neuron, we need to zscore data (i.e., make data mean 0 and variance 1 for each neuron) 

In [14]:
meanX = np.mean(X, axis = 0);
stdX = np.std(X, axis = 0);
X = (X-meanX)/stdX;

## Divide data to high and low rates
    Divide data to high rate trials and low rate trials

In [70]:
Y_HR = Y[Y==1];
X_HR = X[Y==1,:];
Y_LR = Y[Y==0];
X_LR = X[Y==0,:];
print ' %d trials are high rate and %d trials are low rate' %(len(Y_HR), len(Y_LR))

 137 trials are high rate and 137 trials are low rate


## Handle inbalance in the number of trials
    - Still need to think about that.

## Identify the best regularization parameter:
    Perform 10-fold cross validation to obtain the best regularization parameter

In [71]:
from crossValidateModel import crossValidateModel
from linearSVM import linearSVM
from compiler.ast import flatten
numSamples = 2;
lvect = flatten([0, list(10**(np.arange(-5, 2,0.5)))]);
print 'try the following regularization values: %\n', lvect

for s = 1:numSamples
    for i = 1:len(lvect):
        summary =  crossValidateModel(X, Y, linearSVM, kfold = 10, l1 = lvect[i])
    
            

SyntaxError: invalid syntax (<ipython-input-71-6ce3f751932b>, line 8)

running l2 svm classification


28.571428571428569

In [ ]:
def crossValidate(X, Y, **options):
    #%%
    import numpy as np
    import matplotlib.pyplot as plt
    from numpy import random as rng
    from compiler.ast import flatten
    from sklearn import svm

    Y = np.squeeze(np.array(Y).astype(int));
    
    if X.shape[0]>len(Y):
        numObservations = len(Y);
        numFeatures = len(X)/numObservations;
        X = np.reshape(np.array(X.astype('float')), (numObservations, numFeatures), order='F');
        
    scale = np.sqrt((X**2).mean());
    mskOutNans = (np.sum(np.isnan(X), axis = 1)+ np.squeeze(np.isnan(Y)))<1 ;
    X = X[mskOutNans, :];
    Y = Y[mskOutNans];  
    numObservations, numFeatures = X.shape;
    lvect = flatten([0, list(10**(np.arange(-5, 2,0.5)))]);            
    l = np.array(lvect*scale);
    
    perClassErrorTest = np.nan+np.ones((numSamples, l.shape[0]));
    perClassErrorTrain = np.nan+np.ones((numSamples, l.shape[0]));
    
    for s in range(numSamples):
        ## %%%%%% shuffle trials to break any dependencies on the sequence of trails 
        shfl = rng.permutation(np.arange(0, numObservations));
        Ys = Y[shfl];
        Xs = X[shfl, :]; 
            
        ## %%%%% divide data to training and testin sets
        YTrain = Ys[np.arange(0, int((kfold-1.)/kfold*numObservations))];
        YTest = Ys[np.arange(int((kfold-1.)/kfold*numObservations), numObservations)];
            
        XTrain = Xs[np.arange(0, int((kfold-1.)/kfold*numObservations)), :];
        XTest = Xs[np.arange(int((kfold-1.)/kfold*numObservations), numObservations), :];
        ## %%%%% loop over the possible regularization values
        for i in range(l.shape[0]):
            # Create a classifier: a support vector classifier
            if regType == 'l1':
                linear_svm = svm.LinearSVC(C = 1./l[i], loss='squared_hinge', penalty='l1', dual=False)
            elif regType == 'l2':
                linear_svm = svm.LinearSVC(C = 1./l[i], loss='squared_hinge', penalty='l2', dual=False)

            linear_svm.fit(XTrain, np.squeeze(YTrain))
            perClassErrorTest[s, i] = predictErrorSVM(YTest, linear_svm.predict(XTest));
            perClassErrorTrain[s, i] = predictErrorSVM(YTrain, linear_svm.predict(XTrain));

        print 'cross-validating: %.2f %% completed' % ((s+1.)/(numSamples+0.)*100.) 
    
    meanPerClassErrorTrain = np.mean(perClassErrorTrain, axis = 0);
    semPerClassErrorTrain = np.std(perClassErrorTrain, axis = 0)/np.sqrt(numSamples);
    
    meanPerClassErrorTest = np.mean(perClassErrorTest, axis = 0);
    semPerClassErrorTest = np.std(perClassErrorTest, axis = 0)/np.sqrt(numSamples);
    ix = np.argmin(meanPerClassErrorTest);
    l = l[meanPerClassErrorTest <= (meanPerClassErrorTest[ix]+semPerClassErrorTest[ix]), :];
    lbest = l[-1, :]; # best regularization term based on minError+SE criteria
    ix = np.sum(l==lbest,1)==2
    ##%%%%%% plot coss-validation results
    plt.figure('cross validation')
    
    plt.fill_between(lvect, meanPerClassErrorTrain-semPerClassErrorTrain, meanPerClassErrorTrain+ semPerClassErrorTrain, alpha=0.5, edgecolor='k', facecolor='k')
    plt.fill_between(lvect, meanPerClassErrorTest-semPerClassErrorTest, meanPerClassErrorTest+ semPerClassErrorTest, alpha=0.5, edgecolor='r', facecolor='r')
    plt.plot(lvect, meanPerClassErrorTrain, 'k', label = 'training')
    plt.plot(lvect, meanPerClassErrorTest, 'r', label = 'validation')
    plt.plot(np.array(lvect)[ix], meanPerClassErrorTest[ix], 'bo')
    plt.xlim([lvect[1], lvect[-1]])
    plt.xscale('log')
    plt.xlabel('regularization parameter')
    plt.ylabel('classification error (%)')
    plt.legend()
    return lbest
    


Run lasso SVM:

In [17]:
from lassoLinearSVM import crossValidateSVM

kfold = 10
numSamples = 2
lbest = crossValidateSVM(X, Y, 'l1', kfold, numSamples)


ImportError: cannot import name crossValidateSVM